In [12]:
from box import ConfigBox
from ruamel.yaml import YAML
from sagemaker.pytorch.estimator import PyTorch

In [22]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()

role = sagemaker.get_execution_role()

role

'arn:aws:iam::342840881361:role/service-role/AmazonSageMaker-ExecutionRole-20220507T201599'

In [8]:
%env SM_MODEL_DIR=~/example-get-started-experiments-non-demo/models
%env SM_CHANNEL_TRAIN=~/example-get-started-experiments-non-demo/data/train_data

env: SM_MODEL_DIR=~/example-get-started-experiments-non-demo/models
env: SM_CHANNEL_TRAIN=~/example-get-started-experiments-non-demo/data/train_data


In [17]:
yaml = YAML(typ="safe")
params = ConfigBox(yaml.load(open("../params.yaml", encoding="utf-8")))
args = params["base"] | params["train"]
args["fine_tune_args.epochs"] = args["fine_tune_args"]["epochs"]
args["fine_tune_args.base_lr"] = args["fine_tune_args"]["base_lr"]
del args["fine_tune_args"]
args

ConfigBox({'random_seed': 42, 'valid_pct': 0.1, 'arch': 'shufflenet_v2_x2_0', 'img_size': 256, 'batch_size': 8, 'fine_tune_args.epochs': 8, 'fine_tune_args.base_lr': 0.005})

In [25]:
pytorch_estimator = PyTorch('../src/train.py',
                            instance_type='ml.g4dn.xlarge',
                            instance_count=1,
                            framework_version='2.0.0',
                            py_version='py310',
                            hyperparameters = args,
                            role=role,
                            source_dir='../src')
pytorch_estimator.fit({'train': 's3://example-get-started-experiments-non-demo/data/train_data'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-07-14-21-29-35-387


Using provided s3_resource
2023-07-14 21:29:35 Starting - Starting the training job...
2023-07-14 21:29:52 Starting - Preparing the instances for training......
2023-07-14 21:30:57 Downloading - Downloading input data......
2023-07-14 21:31:38 Training - Downloading the training image........................
2023-07-14 21:35:49 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-14 21:35:59,255 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-14 21:35:59,273 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-14 21:35:59,282 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-14 21:35:59,288 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-07-14 21:36:01,302 sagemaker-training-toolki

UnexpectedStatusException: Error for Training job pytorch-training-2023-07-14-21-29-35-387: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 2
ErrorMessage ""
Command "/opt/conda/bin/python3.10 ../src/train.py --arch shufflenet_v2_x2_0 --batch_size 8 --fine_tune_args.base_lr 0.005 --fine_tune_args.epochs 8 --img_size 256 --random_seed 42 --valid_pct 0.1", exit code: 1